## Deliverable 3. Create a Travel Itinerary Map.

In [69]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [80]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("/Users/rctay/Desktop/DU Data Class/World_weather_analysis/Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Bluff,NZ,60.80,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
1,1,Camacha,PT,60.22,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
2,2,Mount Gambier,AU,54.97,light rain,-37.8333,140.7667,The Commodore
3,3,New Norfolk,AU,51.30,light rain,-42.7826,147.0587,The Shingles Riverside Cottages
4,6,Airai,TL,67.75,scattered clouds,-8.9266,125.4092,NaN
5,8,Olafsvik,IS,38.12,overcast clouds,64.8945,-23.7142,Hotel Olafsvik
6,11,Port Elizabeth,ZA,61.32,clear sky,-33.9180,25.5701,39 On Nile Guest House
7,12,Muros,ES,58.15,scattered clouds,42.7762,-9.0603,Apartment Av. Castelao
8,13,Bethel,US,53.91,moderate rain,41.3712,-73.4140,Courtyard by Marriott Danbury
9,18,Kodiak,US,45.91,overcast clouds,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center


In [81]:
vacation_df.isnull().sum() 

City_ID                 0
City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             47
dtype: int64

In [82]:
vacation_df = vacation_df.dropna()

In [83]:
vacation_df.isnull().sum() 

City_ID                0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [84]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</b></dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_df = vacation_df

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]

In [85]:
# 4a. Add a marker layer for each city to the map.


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)



# 4b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

In [86]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df["City"] == "Portland"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Portland"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Grants Pass"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Pacific Grove"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Pendleton"] 
vacation_start
vacation_stop1
vacation_stop2
vacation_stop3

    

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
35,67,Pendleton,US,60.62,overcast clouds,45.6721,-118.7886,Americas Best Value Inn Pendleton


In [87]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')

stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3: {stop3}')

Start: (45.5234, -122.6762), End: (45.5234, -122.6762)
Stop 1: (42.439, -123.3284), Stop 2: (36.6177, -121.9166), Stop 3: (45.6721, -118.7886)


In [101]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary_Driving = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode="DRIVING")


city_itinerary_Walking = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='WALKING')
city_itinerary_Biking = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='BICYCLING')

fig.add_layer(city_itinerary_Driving)

#fig.add_layer(city_itinerary_Biking)

#fig.add_layer(city_itinerary_Walking)

fig


Figure(layout=FigureLayout(height='420px'))

In [89]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3,],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,282,Portland,US,60.53,moderate rain,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
1,515,Grants Pass,US,64.90,broken clouds,42.4390,-123.3284,La Quinta Inn & Suites by Wyndham Grants Pass
2,608,Pacific Grove,US,70.02,few clouds,36.6177,-121.9166,Lovers Point Inn
3,67,Pendleton,US,60.62,overcast clouds,45.6721,-118.7886,Americas Best Value Inn Pendleton


In [94]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</b></dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [99]:
# 11a. Add a marker layer for each city to the map.


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))